<font color='orange'>**TODO**:

- upstream: 

    1. set up for streaming data manipulation
    2. native data partitioning 
    
- review queries

- downstream:

    1. Technical note series ("Data loading: CCP storage", "Efficient transfer to BQ" ,"Query BQ")
    2. PatStat at a Glance Series
    3. Topics
    
- Earliest filing ids and Auth -> where does the innovation come from ? or go deeper (inventors' adress)     
</font>    


**What is PATSTAT?**

"Patent statistics are used as indicators of the inventive activity of companies or countries ($\ldots$). While the patent activity rose substantially over the last decades, the demand for patent data and statistics followed the same trend. ($\ldots$).

Upon request of the [OECD Statistics] Task Force, the EPO [European Patent Organization] has created the "Worldwide Patent Statistical Database" [(aka PATSTAT)]." (`Data Catalog for Patstat`, 5.07, "1.2 About Patstat)

PATSTAT is built as a relational database with multiple tables related by "keys". Each table consists in a set of columns (equivalently variables) and can be related to other table through a key (primary or foreign, denoted by the suffix `_id`). 

**What is this notebook for?**

The Application table (`TLS201_APPLN`) is PATSTAT's central table. It contains the key bibliographical data elements relevant to identify the patent application. Most of the elements in this table can be found on the first page of a printed patent document. It also links to many other database tables.

This notebook documents the 101 of the Patstat "Application" table (aka `TLS201_APPLN`, `PATSTAT2016a`).

More precisely:

- we report variable descriptions and encoding details from the `Data Catalog for Patstat`(5.07)
- we provide summary statistics and visualization (when relevant)

**Comments**

<font color='orange'>*TODO*</font>

----

**Import and meta**

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from google.cloud import bigquery
from pathora.plots import bar_chart, stacked_bar_chart

import plotly.plotly as py
import plotly.graph_objs as go
import plotly.io as pio
import plotly.figure_factory as ff
import pandas as pd
import os
import pycountry
import colorlover as cl

In [6]:
data_path = 'data/'
plots_path = 'plots/'
views_path = 'views/'

client = bigquery.Client()

----

**TLS201_APPLN Schema**

In [8]:
table_ref = client.dataset('raw').table('tls201_cp')
table = client.get_table(table_ref)
schema = table.schema
schema = [[schema[i].name, schema[i].field_type, schema[i].mode] for i in range(len(schema))]

In [9]:
table = ff.create_table(
    pd.DataFrame(schema, columns=['Name', 'Type', 'Mode']))
py.iplot(table)

/Users/cyrilverluise/.virtualenvs/jupyter/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



----

# Technical identifier

## APPLN_ID



**Full name**: Application identification

**Description**: Technical unique identifier without any business meaning. It refers to the national/regional application.

**Format**: primary Key, , 0 $\ldots$ 999 999 999. 

Note: 

1. Range 1: 1 to 900 000 000. Filed applications which have a related publication in DOCDB. UNique but not sequential.
2. Range 2: from 900 000 001 to 930 000 000. Artificial applications which are created in PATSTAT for prior applications, claimed as *priorities*, which do not have an application-reference in DOCDB.
3. Range 3: 930 000 001 to 960 000 000. Artificial filing applications with kind code `D2` which are created in PATSTAT for those artificial publications which are also created in PATSTAT because these *publications* are cited, but do not have a publication-reference in DOCDB.
3. Range 4: 960 000 001 to 999 999 999. Artificial filing applications with kind code `D3` which are created in PATSTAT because these *applications* are cited. 

```python
query = """
SELECT
  year,
  appln_auth,
  COUNT(*) AS nb_range,
  CASE
    WHEN appln_id BETWEEN 1 AND 900000000 THEN "range_1"
    WHEN appln_id BETWEEN 900000001 AND 930000000 THEN "range_2"
    WHEN appln_id BETWEEN 930000001 AND 960000000 THEN "range_3"
    WHEN appln_id BETWEEN 960000001 AND 999000000 THEN "range_4"
  END AS range
FROM
  raw.tls201_cp
GROUP BY
  year,
  appln_auth,
  nb_range
ORDER BY
  year,
  appln_auth;"""

client.query(query=query).to_dataframe().to_csv(views_path + '201_ApplnId_byRange.csv')
```

In [248]:
df = pd.read_csv(views_path + '201_ApplnId_byRange.csv', index_col=0)

In [249]:
# Overall range distribution
tmp = df.groupby(['range']).sum()['nb_range']

fig = bar_chart(tmp, 'ApplicationId by Range')
pio.write_image(fig, plots_path + '/201_ApplnId_byRange.png')
#py.iplot(fig, filename='201_ApplnId_byRange')

In [243]:
# Range distribution by year
tmp = df.groupby(['year', 'range']).sum()['nb_range'].to_frame().reset_index('range')
fig = stacked_bar_chart(tmp, 'range', 'nb_range', 'ApplnId by range and year')
pio.write_image(fig, plots_path + '/201_ApplnId_byRangeYear.png', width=1200, height=800)
#py.iplot(fig, filename='201_ApplnId_byRange')

# Business identifiers

## APPLN_AUTH

**Full name**: Application Authority

**Description**: Patent Authority where the National, International or Regional application was filed

**Format**: str, ,alpha2 - Mapping provided by `TLS801_country` (TODO: check)

```python

query = """
SELECT 
    appln_auth,
    year,
    count(appln_auth) as nb_patents
FROM
    raw.tls201_cp
GROUP BY 
    appln_auth,
    year
ORDER BY
    appln_auth,
    year;"""

client.query(query=query).to_dataframe().to_csv(views_path + '201_ApplnAuth_byYear.csv')
```

In [278]:
tmp_path = '/201_ApplnAuth_byYearAuth'

In [279]:
df_org = pd.read_csv(data_path + 'tls801_part01.txt')
df = pd.read_csv(views_path + '/201_ApplnAuth_byYear.csv', index_col=0)

In [327]:
# Overall applications by auth since the 1970s
threshold = 1970
tmp = df.query('year>=@threshold').groupby('appln_auth').sum().merge(df_org, how='left',
                                                                     left_index=True, right_on=['ctry_code']).set_index(
    'st3_name')
fig = bar_chart(tmp['nb_patents'].sort_values(ascending=False).iloc[:15],
                'Number of Applications by ApplnAuth since {}'.format(threshold))
pio.write_image(fig, plots_path + '/201_ApplnAuth_byAuth70s.png', width=1200, height=800)
#  py.iplot(fig)


In [301]:
# Application by auth over time
for cnt_code in df.appln_auth.unique():
    try:  # some organizations don't have names ...
        cnt_name = df_org.query('ctry_code == @cnt_code')["st3_name"].values[0]
    except:
        pass
    tmp = df.dropna().set_index('year').query('appln_auth==@cnt_code')['nb_patents']
    # be careful; there is a large number of year=nan for some countries (ex: DE, CA)
    if sum(tmp) > 1e6:
        fig = bar_chart(tmp, 'Applications by year in {}'.format(cnt_name))
        pio.write_image(fig,
                        plots_path + tmp_path + '/201_ApplnAuth_{}_byYear.png'.format(cnt_code),
                        width=1200, height=800)


## APPLN_NR

**Full name**: Application Number

**Description**: Number issued by the Patent Authority where the National, International or Regional application was filed

**Format**: bytes, "", 15bytes

Note: Ends with A, D, K, T or X to create "dummy" application numbers that are present because the number is mandatory but the actual number is not known.

- A: data errors
- D: dummy application; the publication number is put in front of the D
- K: special type of older Brazilian application (number format 11nnnnnK )
- T: dummy technical priority
- X: dummy pre-1970 derived priority



```python
query = """
SELECT
  #  year,
  COUNT(*) AS nb_dummy,
  CASE
    WHEN SUBSTR(appln_nr, -1)="A" THEN "data errors"
    WHEN SUBSTR(appln_nr, -1)="D" THEN "dummy application"
    WHEN SUBSTR(appln_nr, -1)="T" THEN "technical priority"
    WHEN SUBSTR(appln_nr, -1)="X" THEN "pre-1970 derived priority"
    ELSE "no dummy"
  END AS dummy
FROM
  raw.tls201_cp
GROUP BY
  #  year,
  #  appln_auth,
  dummy;
  #ORDER BY
  #  year;
  #  appln_auth;"""
# feel-free to uncomment to get year and appln_auth details
client.query(query=query).to_dataframe().to_csv(views_path + '201_ApplnNR_byDummy.csv')
```

In [329]:
df = pd.read_csv(views_path + '201_ApplnNR_byDummy.csv', index_col=0)

In [336]:
tmp = df.set_index('dummy')['nb_dummy'].sort_values(ascending=False)
fig = bar_chart(tmp, 'ApplnNr by Dummy')
pio.write_image(fig, plots_path + '201_ApplnNR_byDummy.png')

## APPLN_KIND

**Full name**: Kind of application

**Description**: Specification of the kind of application

**Format**: str, , 1-2ascii

Note:

- A: patent
- U: utility model
- F: design patent
- P: provisional application
- W: PCT application (in the international phase)
- T: used by some offices (e. g. AT, DE, DK, ES, GR, HR, PL, PT, SI, SM, TR) for applications which are "translations" of granted PCT or EP applications
- D2, D3: artificial applications
- D,K,L,M,N: dummy for de-duplicating
- Other values are used temporarily to resolve minor problems that would otherwise have prevented the application to be recorded in DOCDB

```python
query = """SELECT
  year,
  appln_auth,
  count(*) AS nb_kind,
  CASE
    WHEN appln_kind="A " THEN "Patent"
    WHEN appln_kind="U " THEN "Utility model"
    WHEN appln_kind="F " THEN "Design patent"
    WHEN appln_kind="P " THEN "Provisional application"
    WHEN appln_kind="W " THEN "PCT application"
    WHEN appln_kind="T " THEN "Translation"
    WHEN appln_kind="D2" OR appln_kind="D3" THEN "Translation"
    WHEN appln_kind="D "
  OR appln_kind="K "
  OR appln_kind="L "
  OR appln_kind="M "
  OR appln_kind="N " THEN "Dummy"
    ELSE "Temp"
  END AS kind
FROM
  raw.tls201_cp
GROUP BY
  year,
  appln_auth,
  kind
ORDER BY
  year,
  appln_auth,
  kind;"""
client.query(query).to_dataframe().to_csv(views_path+'201_ApplnKind_byYearAuth.csv')
```

In [352]:
df = pd.read_csv(views_path + '201_ApplnKind_byYearAuth.csv',
                 index_col=0)


In [356]:
# Overall ApplnKind
tmp = df.groupby('kind').sum()['nb_kind'].sort_values(ascending=False)
fig = bar_chart(tmp, 'Appln kind')
pio.write_image(fig, plots_path + '201_ApplnKind.png')


In [365]:
# ApplnKind by Year
tmp = df.dropna().set_index('year')
fig = stacked_bar_chart(tmp, 'kind', 'nb_kind', title='Appln kind by year')
pio.write_image(fig, plots_path + '201_ApplnKind_byYear.png',
                width=1200, height=800)


<font color=#1F618D>PC: Could look by cnt as well.</font>

## <font color=grey>APPLN_FILING_DATE</font>

<font color=grey>**Full name**: Application filing date

**Description**: Date on which the application was physically received at the Patent Authority

**Format**: date, 9999-12-31, yyyy-mm-dd</font>

##  APPLN_FILING_YEAR

**Full name**: Application filing year

**Description**: Year on which the application was physically received at the Patent Authority

**Format**: date, 9999, yyyy (from APPLN_FLING_DATE)

<font color=#1F618D>PC: Partitioning is derived from the `APPLN_FILING_YEAR`. Many prior queries can help us retrieve the distribution of patents per year at 0 cost.</font>

In [337]:
df = pd.read_csv(views_path + '/201_ApplnAuth_byYear.csv', index_col=0)

In [342]:
tmp = df.groupby('year').sum()['nb_patents']
fig = bar_chart(tmp, title='Appln filing by Year')
pio.write_image(fig, plots_path + '201_ApplnFilingYear_byYear.png',
               width=1200, height=800)

## <font color=grey>APPLN_NR_EPODOC</font>

<font color=grey>**Full name**: EPODOC application number

**Description**: Number in EPODOC format (containing letters and digits) which, if present - will uniquely identify an application. The number is created by the EPO based on the DOCDB application number, application authority and application kind.

**Format**: bytes, "", 20bytes</font>

<font color=#1F618D>PC: can be used to retrieve other variables (see p 96)</font>

## <font color=grey>APPLN_NR_ORIGINAL<font>

<font color=grey>**Full name**: Original application number

**Description**: Application number in original format as provided by the supplier. It is assumed that the number is as printed on the respective publications.

**Format**: bytes, "", 1-100bytes</font>

<font color=grey>Note: Typically these numbers do not contain the country code. In about 10% of the applications no original application number is known.

Comment: This attribute is useful to combine application data of PATSTAT with other databases which also contains the original application number.</font>

## IPR_TYPE

**Full name**: Type of intellectual property

**Description**: Type of Intellectual Property Right

**Format**: str, , 2ascii

Note: Derived from `APPLN_KIND` (see p 169)
    
- PI: Patent of innovation 
- UM: Utility Model
- DP: Design Patent

<font color='orange'>*TODO*</font>: Query by year, auth and ipr_type only.

## <font color=grey>INTERNAT_APPLN_ID</font>

<font color=grey>**Foreign key**

**Full name**: Application identification of the earlier PCT international application for an application

**Description**: Technical unique identifier without any business meaning

**Format**: int, 0, 0 $\ldots$ 999 999 999</font>

# Route of the application

<font color=orange>*TODO*</font>: 

- Summarize : http://www.wipo.int/pct/en/guide/ip03.html#_chapt3 and p 155 (5.07)
- Funnel chart ? From national to international even if the route goes the other way round. Still don't know what to do with that info.


## INT_PHASE

**Full name**: Indicator whether the application is or has been in the international phase

**Description**: Indicates that an application is or has been in the international phase.

**Format**: bool, `False`, {`True`;`False`}

Note: These indicators provide a somewhat *simplistic* approach to identify the route an application has taken. This is the result of interpretations and assumptions for which no responsibility whatsoever can be accepted.

WARNING: These indicators only help to understand applications which actually exist in PATSTAT. It does not help to answer questions like “How many EP applications are valid in country x”, because not every office publishes patents which are validated / granted in their country. Consequently, there is no publication or application in PATSTAT for every granted patent. The same will apply for the Unitary Patents, if there is no publication for that (see p 154).

## REG_PHASE

**Full name**: Indicator whether the application is or has been in the regional phase

**Description**: Indicates that an application is or has been in the regional phase.

**Format**: bool, `False`, {`True`;`False`}

## NAT_PHASE

**Full name**: Indicator whether the application is or has been in the national phase

**Description**: Indicates that an application is or has been in the national phase.

**Format**: bool, `False`, {`True`;`False`}

In [368]:
query="""SELECT
  year,
  appln_auth,
  int_phase,
  reg_phase,
  nat_phase,
  COUNT(*) AS nb_phase
FROM
  raw.tls201_cp
GROUP BY
  year,
  appln_auth,
  int_phase,
  reg_phase,
  nat_phase
ORDER BY
  year,
  appln_auth;"""

client.query(query).to_dataframe().to_csv(views_path + '201_Phases.csv')

In [369]:
df = pd.read_csv(views_path + '201_Phases.csv', index_col=0)

In [384]:
df.groupby(['int_phase', 'reg_phase', 'nat_phase']).sum()['nb_phase'].to_frame()

nb_phase
int_phase reg_phase nat_phase          
False     False     False          5626
                    True       72110968
          True      False       1765314
                    True        1853271
True      False     False       2798628
                    True        4971398
          True      False       1399161
                    True         877567

In [378]:
df.groupby(['int_phase']).sum()

,year,nb_phase
int_phase,,
False,14821520.0,75740125
True,8615728.0,10050063


In [379]:
df.groupby(['reg_phase']).sum()

,year,int_phase,nb_phase
reg_phase,,,
False,21693742.0,3963.0,79888810
True,1509291.0,320.0,5895313


In [380]:
df.groupby(['nat_phase']).sum()

,year,int_phase,nb_phase
nat_phase,,,
False,4526514.0,2076.0,5968729
True,18860694.0,2254.0,79819269


# Data from priorities

## <font color=grey>EARLIEST_FILING_DATE</font>

<font color=grey>**Full name**: Date of the earliest filing

**Description**: The earliest date of the filing dates of the application itself, its Paris Convention priority applications, the applications with which it is related via technical relations and its application continuations.

**Format**: date, 9999-12-31, yyyy-mm-dd
</font>

<font color=grey>Note: Only directly related applications are considered; this is unlike the INPADOC family, where applications might also be indirectly related.</font>

## EARLIEST_FILING_YEAR

**Full name**: Year of the earliest filing date

**Description**: Year of the earliest filing date

**Format**: date, 9999, yyyy

## <font color=grey>EARLIEST_FILING_ID</font>

<font color=grey>**Foreign Key**

**Full name**: Application ID of the earliest filing

**Description**: The ID of the earliest application, considering the application itself, its Paris Convention priority applications, the applications with which it is related via technical relations and its application continuations.

**Format**: int, , 0 $\ldots$ 999 9999 999</font>

<font color=grey>Note: If multiple applications have been filed on the earliest filing date, then conceptually any of these applications can be regarded as the earliest application. Nevertheless, the logic to determine the application which has been filed first is like this:

1. If there is a PCT application which was filed on the earliest application date, then the APPLN_ID of this PCT application is taken as the EARLIEST_FILING_ID.
2. Else: If there are 1 or more Paris convention priorities which were filed on the earliest application date, then the Paris convention priority with the smallest APPLN_ID is taken as the EARLIEST_FILING_ID.
3. Else: the application which was filed on the earliest application date with the smallest APPLN_ID will be taken.
</font>

# Data from publications

<font color=orange>TODO</font>: 

- count by YEAR and AUTH
- avg(year) between EARLIEST_FILING_DATE and EARLIEST_PUBN_DATE

## EARLIEST_PUBLN_DATE

**Full name**: Date of earliest publication

**Description**: Date of earliest publication

**Format**: date, 9999-12-31, yyyy-mm-dd

## EARLIEST_PUBLN_YEAR

**Full name**: Year of earliest publication

**Description**: Year of earliest publication

**Format**: date, 9999, yyyy

## EARLIEST_PAT_PUBLN_ID

**Foreign Key**

**Full name**: ID of the earliest publication

**Description**: The ID of a publication published on the earliest publication date of an application

**Format**: int, 0, 0 $\ldots$ 999 999 999

# Data derived from publications

<font color=orange>TODO</font>: 

- count GRANTED by YEAR and AUTH

## GRANTED

**Full name**: "Granted" indicator

**Description**: True if there exists a publication of the grant; False otherwise

**Format**: bool, , {`True`, `False`}

Note: This variable is the result of interpretations and assumptions for which no responsibility whatsoever can be accepted.

# Family data

## DOCDB_FAMILY_ID

**Foreign Key**

**Full name**: Identifier of a DOCDB simple family

**Description**: Means that *most probably* the applications share exactly the same priorities (Paris Convention or technical relation or others) as in table `TLS201_APPLN`, `TLS204_PRIOR_APPLN`, `TLS205_TECH_REL` and `TLS216_APPLN_CONTN`.

**Format**: int, , 0 $\ldots$ 999 999 999

Note: 0 indicates that the application does not belong to any DOCDB family. This is only the case for the dummy application (APPLN_ID = 0) and for artificial applications (APPLN_ID ≥ 900 000 000). See more p 132.

## INPADOC_FAMILY_ID

**Foreign Key**

**Full name**: Identifier of an INPADOC *extended* priority family

**Description**: Means that the applications share a priority directly or indirectly via a third application. A 'priority' in this case means a link shown between applications as in tables `TLS201_APPLN` (regional/national phase of a PCT application), `TLS204_APPLN_PRIOR` (PARIS convention priorities), `TLS205_TECH_REL` (patents which have been technically linked by patent examiners on the basis of similar content) and table `TLS216_APPLN_CONTN` (continuations, divisions etc.).

**Format**: int, 0, 0 $\ldots$ 999 999 999

Note: Much patent research is affected by the “family” concepts. There are various definitions of how to link different patents into “families”. This INPADOC extended priority family was developed by the INPADOC organisation before it was integrated into the EPO.

## DOCDB_FAMILY_size

**Full name**: Size of DOCDB simple family

**Description**:  Size of DOCDB simple family

**Format**: int, , 0 $\ldots$ 200

<font color=orange>TODO</font>: 

- distribution (requires external info to trace the earliest filing appn such as YEAR and AUTH)

## NB_CITING_DOCDB_FAM

**Full name**: Number of forward citations on family level

**Description**: Number of distinct DOCDB simple families citing at least one of the publications or applications of the DOCDB simple family of the current application (search report citations from `TLS212_CITATION`)

**Format**: int, , 0 $\ldots$ 200

<font color=orange>TODO</font>: 

- distribution (requires external info to trace de earliest filing appn such as YEAR and AUTH)
- interactions between family size and nb_citing (dist plot) https://plot.ly/ipython-notebooks/2d-kernel-density-distributions/

# Aggregated data

## NB_APPLICANTS

**Full name**: Number of applicants of an application

**Description**: Number of applicants of an application *according to the most recent publication*

**Format**: int, , 0 $\ldots$ 

<font color=orange>TODO</font>: distribution + Think about longitudinal dimension (YEAR, AUTH)


<font color=red>WARNING</font>: How to avoid duplicates ? -> Group by earliest_pat_publn_date, DOCD family.

## NB_INVENTORS

**Full name**: Number of inventors of an application

**Description**: Number of inventors of an application *according to the most recent publication*

**Format**: int, , 0 $\ldots$

<font color=orange>TODO</font>: distribution + Think about longitudinal dimension (YEAR, AUTH)


<font color=red>WARNING</font>: How to avoid duplicates ? -> Group by earliest_pat_publn_date, DOCD family.

# Time series of patent application

# IPR type

In [67]:
df_ipr = pd.read_csv(data_path + '/ipr_yr_auth.csv', index_col=0)

In [72]:
df_ipr.query('appln_auth =="FR"').head()

,ipr_type,date,appln_auth,nb_ipr
3524,UM,2015.0,FR,87
3525,PI,2015.0,FR,2292
3526,UM,2014.0,FR,297
3527,PI,2014.0,FR,12236
3528,UM,2013.0,FR,486


In [146]:
def plot_ipr_type(df: pd.DataFrame, cnt: str, path: str = None):
    tmp = df_ipr.query('appln_auth == @cnt').dropna()
    try:
        cnt_name = pycountry.countries.get(alpha_2=cnt).name
    except:
        cnt_name = cnt_a2_name[cnt]
    data = []
    i = 0
    for ipr in tmp.groupby('ipr_type').sum()["nb_ipr"].sort_values(
            ascending=False).index.unique():
        data += [
            go.Bar(
                x=tmp.query('ipr_type == @ipr')["date"].values,
                y=tmp.query('ipr_type == @ipr')["nb_ipr"].values,
                name=ipr,
                marker=dict(color=cl.flipper()['seq']['3']['Reds'][i], ))
        ]
        i += 1
    layout = go.Layout(
        barmode='stack',
        title='Composition of patent applications in {}'.format(cnt_name))
    fig = go.Figure(data=data, layout=layout)
    pio.write_image(fig, path + '/{}_ts_patents.png'.format(cnt))
    #py.iplot(fig, filename='stacked-bar', )

In [147]:
no_match = []
for cnt in df_ipr.appln_auth.unique():
    if df_pat_yrcnt.query(
            'appln_auth == @cnt')['nb_patents'].dropna().sum() > 1e5:
        try:
            plot_ipr_type(df_ipr, cnt, plots_path + 'ipr_type/')
        except:
            no_match += [cnt]
    else:
        pass

# Granted 

In [154]:
df_granted.dropna().query('appln_auth=="FR"').head()

,granted,year,appln_auth,nb_grant
174,False,1844.0,FR,1
188,False,1858.0,FR,1
195,False,1861.0,FR,6
210,False,1866.0,FR,1
238,False,1875.0,FR,1


In [184]:
def plot_grant(df: pd.DataFrame, cnt: str, path: str = None):
    tmp = df.query('appln_auth == @cnt').dropna()
    try:
        cnt_name = pycountry.countries.get(alpha_2=cnt).name
    except:
        cnt_name = cnt_a2_name[cnt]
    data = []

    for boo in [True, False]:
        i = 0 if boo == False else -1
        data += [
            go.Bar(
                x=tmp.query('granted == @boo')["year"].values,
                y=tmp.query('granted == @boo')["nb_grant"].values,
                name=boo,
                marker=dict(color=cl.scales['3']['div']['RdBu'][i], ))
        ]
    layout = go.Layout(
        barmode='stack', title='Patent grants in {}'.format(cnt_name))
    fig = go.Figure(data=data, layout=layout)
    pio.write_image(fig, path + '/{}_ts_grants.png'.format(cnt))


In [185]:
no_match = []
for cnt in df_granted.appln_auth.unique():
    if df_pat_yrcnt.query(
            'appln_auth == @cnt')['nb_patents'].dropna().sum() > 1e5:
        try:
            plot_grant(df_granted, cnt, plots_path + 'pat_grant/')
        except:
            no_match += [cnt]
    else:
        pass

In [174]:
plot_grant(df_granted, "FR", plots_path + 'pat_grant/')

There might be something wrong with the way the `granted` var is coded. Exhibits strange behaviors. Might be due to articificial applications. Check using appln_kind.

# Snippets

## Stacked chart

def get_trace(df: pd.DataFrame, cnt:str):
    tmp = df.query('appln_auth == @cnt').sort_values('year')
    return [go.Bar(
            x=tmp['year'].values,
            y=tmp['nb_patents'].values,
            name = cnt)]
            
data = []
for cnt in df_pat_yrcnt.appln_auth.unique():
    data += get_trace(df_pat_yrcnt, cnt)

#data = [trace for trace in trace.values()]
layout = go.Layout(barmode='stack')

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='stacked-bar')